<a href="https://colab.research.google.com/github/Sabelz/Master_Thesis_Alexander/blob/main/functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Functions and other utils for the project

# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Master_Thesis_Alexander
!git config --global user.email "alexander.sabelstrom.1040@student.uu.se"
!git config --global user.name "Sabelz"

import numpy as np
import matplotlib.pyplot as plt
import torch
!pip install gpytorch
import gpytorch

Mounted at /content/drive
/content/drive/MyDrive/Master_Thesis_Alexander
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 9.6 MB/s eta 0:00:00


# Training Function

In [2]:
def train(model, likelihood, train_x, train_y, training_iter=10):
    """
    This function trains a Gaussian Process (GP) model using the Adam optimizer
    and the Exact Marginal Log Likelihood (MLL) loss.
    The function first sets up the optimizer and the MLL. Then,
    for each training iteration, it zeros out the gradients from the previous iteration,
    computes the model's output on the training data, calculates the loss,
    backpropagates the gradients, and updates the model's parameters with a step of the optimizer.

    Parameters:
    model (gpytorch.models.GP): The GP model to train.
    likelihood (gpytorch.likelihoods.Likelihood): The likelihood associated with the model.
    train_x (torch.Tensor): The training inputs.
    train_y (torch.Tensor): The training targets.
    training_iter (int, optional): The number of training iterations. Defaults to 10.

    Returns:
    None
    """
    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_x)
        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()
        optimizer.step()


# Predict Function

In [10]:
def predict(model, likelihood, test_x):
    """
    This function makes predictions using a given model and likelihood.

    The function sets the model and likelihood to evaluation mode,
    then computes the likelihood of the model's predictions on the test data.
    It uses PyTorch's `no_grad` context manager to avoid tracking gradients during the prediction,
    and GPyTorch's `fast_pred_var` setting for efficient computation.

    Parameters:
    model (gpytorch.models.GP): The Gaussian Process model to make predictions with.
    likelihood (gpytorch.likelihoods.Likelihood): The likelihood associated with the model.
    test_x (torch.Tensor): The test inputs to make predictions on.

    Returns:
    gpytorch.distributions.MultivariateNormal: The distribution of the model's predictions.
    """
    model.eval()
    likelihood.eval()
    # Make predictions by feeding model through likelihood
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        return likelihood(model(test_x))

In [9]:
!git push

remote: Permission to Sabelz/Master_Thesis_Alexander.git denied to Sabelz.
fatal: unable to access 'https://github.com/Sabelz/Master_Thesis_Alexander.git/': The requested URL returned error: 403
